In [1]:
import pandas as pd
import numpy as np
import rasterio
from pyproj import Transformer, CRS

# --- 1. Завантажуємо початковий датасет з точками (він уже містить x, y, x_merc, y_merc та cls_nmad_fab)
df = pd.read_parquet("data/NMAD_with_embeddings_cls.parquet")

# Яка CRS у ваших координат?
# x,y зазвичай EPSG:4326 (довгота/широта), x_merc,y_merc — EPSG:3857
POINTS_CRS = CRS.from_epsg(3857)   # ми беремо x_merc/y_merc
px = df["x_merc"].to_numpy()
py = df["y_merc"].to_numpy()

def sample_raster_auto(path: str,
                       x_in: np.ndarray,
                       y_in: np.ndarray,
                       src_crs: CRS) -> np.ndarray:
    """
    Семплить растр по масиву точок в їхній CRS; якщо CRS відрізняється — трансформує до CRS растра.
    Повертає np.ndarray з float, NoData -> np.nan.
    """
    with rasterio.open(path) as src:
        rast_crs = CRS.from_wkt(src.crs.to_wkt()) if src.crs else None
        if rast_crs is None:
            raise ValueError(f"Raster {path} has no CRS")

        # координати, в CRS растра
        if rast_crs != src_crs:
            transformer = Transformer.from_crs(src_crs, rast_crs, always_xy=True)
            xr, yr = transformer.transform(x_in, y_in)
        else:
            xr, yr = x_in, y_in

        # вибірка
        vals = np.full(shape=(x_in.shape[0],), fill_value=np.nan, dtype="float32")
        for i, (xx, yy) in enumerate(zip(xr, yr)):
            v = next(src.sample([(xx, yy)]), [np.nan])[0]
            # NoData → NaN
            if src.nodata is not None and np.isclose(v, src.nodata):
                vals[i] = np.nan
            else:
                vals[i] = v
        return vals

# --- 2. Опис, що саме додаємо (шляхи підстав свої)
rasters = {
    # aspect (в градусах 0..360)
    "aspect_deg":  r"D:\paper_data\DEM_DATA\data\DATA_UTM\dem\terrain_attributes\fab_dem_utm32635_aspect.tif",
    # кривизна/шорсткість
    "curvature":   r"D:\paper_data\DEM_DATA\data\DATA_UTM\dem\terrain_attributes\fab_dem_utm32635_curvature.tif",
    "roughness":   r"D:\paper_data\DEM_DATA\data\DATA_UTM\dem\terrain_attributes\fab_dem_utm32635_roughness.tif",
    # позиційні та індекси складності рельєфу
    "tpi":         r"D:\paper_data\DEM_DATA\data\DATA_UTM\dem\terrain_attributes\fab_dem_utm32635_tpi.tif",
    "tri":         r"D:\paper_data\DEM_DATA\data\DATA_UTM\dem\terrain_attributes\fab_dem_utm32635_tri.tif",
    # гідро-похідні (за наявності)
    "d8_accum":    r"D:\paper_data\DEM_DATA\data\DATA_UTM\dem\flow_TWI\fab_dem_utm32635_d8_accum.tif",
    # приклад BREACHED DEM (інколи згодиться як додаткова висота)
    "fab_breached": r"D:\paper_data\DEM_DATA\data\DATA_UTM\dem\breached_dems\fab_dem_utm32635_breached.tif",
}

# --- 3. Семплимо всі растрі та додаємо колонки
for col, path in rasters.items():
    print(f"Sampling {col} from {path}")
    df[col] = sample_raster_auto(path, px, py, POINTS_CRS)

# --- 4. Коректне кодування aspect: sin/cos і забудь про 0/360 розрив
# деякі алгоритми пишуть flat = -1 — замінимо на NaN -> 0 після sin/cos
a = df["aspect_deg"].copy()
a = a.where(a >= 0)  # -1 -> NaN
a_rad = np.deg2rad(a)
df["aspect_sin"] = np.sin(a_rad).fillna(0.0)
df["aspect_cos"] = np.cos(a_rad).fillna(0.0)
df.drop(columns=["aspect_deg"], inplace=True)

# (опційно) Лог-скейл для d8_accum, щоб «розтягнути» динаміку
if "d8_accum" in df.columns:
    df["d8_accum_log1p"] = np.log1p(df["d8_accum"])

# --- 5. Збереження оновленого датасету
out_path = "data/NMAD_with_embeddings_cls_features.parquet"
df.to_parquet(out_path, index=False)
print("Saved ->", out_path)


Sampling aspect_deg from D:\paper_data\DEM_DATA\data\DATA_UTM\dem\terrain_attributes\fab_dem_utm32635_aspect.tif
Sampling curvature from D:\paper_data\DEM_DATA\data\DATA_UTM\dem\terrain_attributes\fab_dem_utm32635_curvature.tif
Sampling roughness from D:\paper_data\DEM_DATA\data\DATA_UTM\dem\terrain_attributes\fab_dem_utm32635_roughness.tif
Sampling tpi from D:\paper_data\DEM_DATA\data\DATA_UTM\dem\terrain_attributes\fab_dem_utm32635_tpi.tif
Sampling tri from D:\paper_data\DEM_DATA\data\DATA_UTM\dem\terrain_attributes\fab_dem_utm32635_tri.tif
Sampling d8_accum from D:\paper_data\DEM_DATA\data\DATA_UTM\dem\flow_TWI\fab_dem_utm32635_d8_accum.tif
Sampling fab_breached from D:\paper_data\DEM_DATA\data\DATA_UTM\dem\breached_dems\fab_dem_utm32635_breached.tif
Saved -> data/NMAD_with_embeddings_cls_features.parquet
